### Import

---



In [8]:
import numpy as np
import random as rand
from note_include.elem.Ring   import Ring
from note_include.elem.LWE    import LWE
from note_include.elem.RLWE   import RLWE
from note_include.elem.RLWEp  import RLWEp
from note_include.elem.RGSW   import RGSW
from note_include.FHEW        import FHEW
from note_include.utils.types import RGSWctxt, RLWEctxt, RLWEpctxt, LWEctxt

np.set_printoptions(threshold=np.inf, linewidth=np.inf)

---

# DM-like blind rotation

* <a href="https://eprint.iacr.org/2014/816"> FHEW: Bootstrapping Homomorphic Encryption in less than a second </a>
* <a href="https://eprint.iacr.org/2020/086"> Bootstrapping in FHEW-like Cryptosystems </a>

시작하기 앞서 FHEW 의 핵심인 blind rotation 을 수행하는 알고리즘은 크게 세 가지가 있다.

1. DM-like blind rotation

2. CGGI-like blind rotation

3. LMKCDEY-like blind rotation

이 중 우리는 DM-like blind rotatino 에 대해서 다룰 것이며, 추후에 CGGI, LMKCDEY 또한 다루어보록 하겠다.

---

본격적으로 FHEW 에 대해서 알아보자. 우선 FHEW 는 message 로 비트를 가지며 동형 논리연산을 지원하는 동형암호이다.

메세지의 영역이 $m \in \{0, 1\}$ 로 정의되어서 매우 작은 노이즈에도 메세지가 지워져버리게 된다.

이를 위해 FHEW 에서는 scaling factor 를 메세지에 임의로 곱해주어서 사용하게 되며, FHEW 암호화와 복호화는 다음과 같이 정의할 수 있다.

>$\textbf{Definition}$
>
>Let $m \in \{0,1\}$ is a message, $\vec{a}$ is a vector of random number $a_i \in \Z_q$, $\vec{s}$ is a secret values, and $e$ is a small noise value.   
>Then we can define encryption method as follows:
>
>\begin{align*}
>    \textsf{FHEW.enc}(m; \vec{s}) = (\vec{a}, b), \text{ where } b = \lang \vec{a}, \vec{s} \rang + m \cdot q/4 + e.
>\end{align*}
>
>The decryption method can be easily defined as follows:
>
>\begin{align*}
>    \textsf{FHEW.dec}((a,b); \vec{s}) = b - \lang \vec{a}, \vec{s} \rang = m \cdot q/4 + e \approx m \cdot q/4.
>\end{align*}

코드를 통해 알아보자.

In [2]:
def fhew_encrypt(m:int, s:list[int], CC:LWE) -> LWEctxt:
    assert m == 0 or m == 1, "message must be 0 or 1"
    return CC.encrypt(m * (CC.q // 4), s)


---

그럼 이러한 암호문들을 갖고 어떻게 동형 논리연산을 지원하는지에 대해서 알아보자.

다음 사진은 FHEW 의 동형 논리연산 과정을 보여주고 있다.

![Blind rotation procedure](../images/blind-rotation-procedure.png)

Mod switch 와 key switch 는 이전에 다루었으며, 우리가 이번에 배우고자 하는 과정은 크게 세가지이다.

1. ACC initialization

2. Blind rotation

3. LWE extraction

하나 하나 밟아나가 보자.

우선 두 암호문에 대해서 $(\odot)$ 연산을 수행하게 되는데, 이는 수행하고자 하는 동형 논리연산에 따라 각기 다르게 정의된다. 이에 대한 표는 다음을 참고하자.

![Mapping function](../images/mapping-function.png)

주의할 것은 앞서 정의했던 RGSW 와 RLWE 간의 곱인 $(\odot)$ 연산 과는 다르다. 표기의 혼동을 피하기 위해 이번 장에는 이를 $(\odot')$ 으로 표기하도록 하겠다.

---

## 1. ACC initialization

두 암호문에 대해서 $(\odot)$ 연산을 수행한 이후에 ACC initialization 이라는 연산을 수행하게 된다.

우선 ACC initialization 의 정의부터 확인하고 가자.

>$\textbf{Definition : ACC initialization}$
>
>LWE 암호문 $(\vec{a}, b) \in \mathbb{Z}_q^n$ 이 주어질 때, ACC initialization 은 다음과 같이 정의된다:
>
>\begin{align*}
>    \textsf{ACC\_init}(\vec{a}, b) = (\boldsymbol{0}, \sum_{i=0}^{q/2-1} f(b-i)X^i) = \textsf{acc} \in \mathcal{R}_Q^2
>\end{align*}
>
>여기서 $f$ 는 mapping function 을 의미한다.

$f$ 는 수행하고자 하는 동형 논리연산에 따라 각기 다르게 정의되며 자세한 정의는 위의 표를 참고하길 바란다.

여기서 우리는 왜 하필 $\sum_{i=0}^{q/2-1} f(b-i)X^i$ 로 정의하는지에 대해서 깊게 고민할 필요가 있다.

논리적 모순이 발생하지만 쉽게 이해하기 위해 $\sum_{i=0}^{q-1} (b-i)X^i$ 로 정의했다고 생각해보자. 그럼 이는 다음과 같이 표현할 수 있다.

\begin{equation*}
    (b-0) + (b-1)X^{1} + (b-2)X^{2} + \cdots + (b-t)X^{t} + \cdots + (b-(q-1))X^{q-1}
\end{equation*}

여기서 한 가지 재미있는 사실을 찾을 수 있다. $t = \langle \vec{a}, \vec{s} \rangle \in \mathbb{Z}_q$ 인 상황을 생각해보자.

그럼 위의 다항식에 차수가 $t$ 인 계수는 무엇인지 생각해보면 $ b - t = b - \langle \vec{a}, \vec{s} \rangle = m + e \approx m$ 이라는 것을 금방 알 수 있다.

여기서 $\textsf{acc}$ 를 $\sum_{i=0}^{q-1} f(b-i)X^i$ 으로 정의하게 되면, 차수가 $t$ 인 계수에는 $f(m+e \approx m)$ 이 오게 된다.

그럼 다시 질문해보자. 우리는 $X^t$ 에 위치한 계수를 어떻게 찾을 수 있을까? 우리는 $\vec{s}$ 를 모르기 때문에 $t$ 를 모르고있는 상황임을 기억하자.

이러한 $X^t$ 에 위치한 계수를 상수항을 끌고오는 알고리즘이 바로 FHEW 의 꽃 blind rotation 이다.

---

## 2. Blind rotation

앞서 우리는 RLWE 의 변형인 RLWE' 과 RGSW 를 정의하여 각각 평문 곱셈, 동형 곱셈을 노이즈 관리 측면에서 효율적으로 수행할 수 있도록 만들었다.

이러한 빌드업을 진행했던 이유가 $X^t$ 에 위치한 계수를 동형 곱셈을 통해 외부에 데이터 유출 없이 상수항을 끌고 오기 위함이었다.

예시를 들어 우리가 만약 $\text{RGSW}(X^{-t})$ 를 갖고있다고 가정해보자.

그럼 우리는 다음과 같은 사실을 알 수 있다.

\begin{align*}
    &\textsf{acc} \odot' \text{RGSW}(X^{-t}) = \text{RLWE}\left( \boldsymbol{P} \cdot X^{-t} \right), \\
    &\text{where } \boldsymbol{P} = \sum_{i=0}^{q-1} f(b-i)X^i.
\end{align*}

엄밀하게 따지자면 약간의 오류가 있기는 하지만 그래도 동형 연산을 통해 $X^{-t}$ 를 $\boldsymbol{P}$ 에 곱하여 $X^t$ 에 위치한 계수를 상수항에 위치하도록 만들 수 있다.

그렇다면 $\text{RGSW}(X^{-t})$ 을 사전에 만들어 두어야 한다는 것을 알 수 있다. 이러한 암호문을 blind rotation key $\textsf{brk}$ 라고 한다.

그러나 $t = \langle \vec{a}, \vec{s} \rangle \in \mathbb{Z}_q$ 이며 $\vec{a}$ 는 연산을 수행할 때 마다 변하는 값이므로 key switching 에서 언급되었던 문제점이 다시 발생한다.

무식하게 모든 경우에 대해 blind rotation key 를 만들어두면 크기가 대략 $\mathcal{O}(q^n \cdot n)$ 이 되어서 너무 커져버린다.

이를 해결하기 위해서 key switching 과 똑같이 모든 $a \in \mathbb{Z}_q$ 에 대해서 수행할 수 있도록 한다.

즉, blind rotation key 는 다음과 같이 정의할 수 있다.

>$\textbf{Definition : Blind rotation key}$
>
>어떠한 base value $B_r$ 과 비밀키 $\vec{s}$ 가 있을 때, blind rotation key, $\textsf{brk}$ 는 다음과 같이 정의할 수 있다:
>
>\begin{align*}
>    \textsf{brk}_{i, j, v} = \left\{ \text{RGSW}(X^{v B_r^j s_i}) | i \in [0, n), j \in [0, d_r), v \in [0, B_r) \right\} 
>\end{align*}
>
>여기서 $d_r = \lceil \log_{B_r}q \rceil$ 이다.

이렇게 정의하면 기존의 암호문의 수 $\mathcal{O}(q^n \cdot n)$ 을 $\mathcal{O}(d_r \cdot B_r \cdot n)$ 으로 줄일 수 있다.

이어서 blind rotation 도 정의해보자.

>$\textbf{Definition : Blind rotation}$
>
>LWE 암호문 $(\vec{a}, b) \in \mathbb{Z}_q^n$ 가 주어지고 $\textsf{acc} = \textsf{ACC\_init}(\vec{a}, b)$ 이라고 해보자.
>
>Blind rotation key $\textsf{brk}$ 가 주어졌을 때 blind rotation 은 다음과 같이 정의된다.
>
>\begin{align*}
>    \textsf{Blind\_rotation}\left( 
>        \textsf{acc}, \textsf{brk}    
>    \right) = \textsf{acc} \odot' \textsf{brk}_{i, j, a_{ij}} \text{ for all } a_i \in \vec{a}.
>\end{align*}
>
>여기서 $(a_{ij}) = \textsf{gadget\_decomposition\_int}(a_i)$ 이다.

코드를 통해 확인해보자!


In [9]:
# def dm_like_rgsw_keygen(self, s:list[int], s_ring:Ring):
#     rgsw_keys = []
#     for _s in s:
#         matrix = []
#         for v in range(self.B):
#             row = []
#             for j in range(self.d_g):
#                 coef = np.zeros(self.N)
#                 tmp = (v * (self.B**j) * _s) % self.q
#                 coef[(self.N//2) - int(tmp)] = 1
#                 poly = Ring(self.N, self.Q, coef)
#                 row.append(self.RGSW_CC.encrypt(poly, s_ring))
#             matrix.append(row)
#         rgsw_keys.append(matrix)
#     return rgsw_keys

def ACC_init(b, q, N):
    acc = np.zeros(N)

    for i in range(q//2):
        acc[i] = b
        b = (b - 1) % q

    acc = Ring(N, q, acc)
    return acc

def Blind_rotation(acc : Ring, a, s, q, n, N):
    tmp_acc = acc
    for i in range(n):
        _as = a[i] * s[i] % q
        if _as == 0 : continue

        reduced = False
        if (_as >= N) :
            _as -= N
            reduced = True

        if _as == 0:
            tmp_acc = -1 * tmp_acc
            continue

        if reduced == False:
            monomial = np.zeros(N)
            monomial[(N)-(_as)] = -1
            monomial = Ring(N, q, monomial)
        else:
            monomial = np.zeros(N)
            monomial[(N)-(_as)] = 1
            monomial = Ring(N, q, monomial)
            
        tmp_acc = tmp_acc * monomial

    return tmp_acc

In [163]:
n = 8
q = 512
N = 256

for i in range(1):
    m1 = rand.randint(0, 50)
    a = np.array([rand.randint(0, q) for _ in range(n)])
    s = np.array([rand.randint(0, 1) for _ in range(n)])

    b = ((m1) + sum(a * s)) % q

    acc = ACC_init(b, q, N)
    res = Blind_rotation(acc, a, s, q, n, N)

    _as = sum(a*s)
    _as = int((((_as % q) + q) % q) * (2 * (N) // q))
    
    reduced = False
    if (_as >= N) :
        _as -= N
    reduced = True

    if sum(a*s) % q < N: # even 
        ideal = acc[_as]
    else: # sum(a*s) % q >= N
        acc = -1 * acc
        ideal = acc[_as]


    print("<a,s>         : ", sum(a * s) % q)
    print("Message       : ", m1)
    print("Constant term : ", res.coeffs[0])
    # print("Ideal result  : ", acc[_as])

    if ideal != res.coeffs[0]:
        print("Error?\n")
        break

<a,s>         :  222
Message       :  0
Constant term :  0



---

여러번 수행하다보면 어떤 경우에는 $m$ 이 정상적으로 나오는 경우도 있고, 어떠한 경우에는 $N-m$ 이 나오는 경우도 있을 것이다.

그 이유는 preliminaries 에서 다루었던 negaqcyclic property 와 파라미터 $q = 2N$ 에 있다.

일반적으로 FHEW 에서는 최적화를 비롯한 여러 이유때문에 $q=2N$ 으로 고정해두어 사용한다.

또한 $\langle \vec{a}, \vec{s} \rangle \in \mathbb{Z}_q$ 이기 때문에 사실은 $ \langle \vec{a}, \vec{s} \rangle = v + n q $ 로 보는것이 더욱 정확하다.

이 때 $q = 2N$ 이기 때문에 여기서는 negacyclic property 를 고려하지 않아도 된다.

그러면 돌아와서 $\langle \vec{a}, \vec{s} \rangle \in \mathbb{Z}_q$ 으로 볼 때, $\langle \vec{a}, \vec{s} \rangle < N$ 일 경우에는 정상적으로 $m$ 이 나오며, 그 외에는 $N-m$ 이 나오는 것이다.

우리는 다시 돌아가서 mapping function 의 정의에 대해서 다시 생각해 볼 필요가 있다.

자세히 보면 mapping function $f$ 가 $f(v) = -f(v + (q/2))$ 를 만족하다는 것을 알 수 있다.

이렇게 설정함을 통해 위와 상수항에 항상 원하는 값이 오는 것을 보장할 수 있다.

---

이어서 mapping function $f$ 가 왜 저런식으로 정의되는지 알아보자.

FHEW 에서 메세지는 $m\in\{0,1\}$ 으로 정의된다. 여기에 암호화 하는 과정에서 scaling factor $q/4$ 를 곱하게 된다.

즉, $m = 0$ 인 경우에는 $0$, $m = 1$ 인 경우에는 $q/4$ 를 암호화 하고 있다는 것이다.

여기서 AND 연산을 예시로 들어보자. 이 때 두 암호문 간의 연산 $(\odot)$ 은 덧셈이며, 이를 바탕으로 진리표를 만들어 보자.

\begin{align*}
    0 \text{ and } 0 &= 0 + 0     &= 0    \\ 
    0 \text{ and } 1 &= 0 + q/4   &= q/4  \\
    1 \text{ and } 0 &= q/4 + 0   &= q/4  \\
    1 \text{ and } 1 &= q/4 + q/4 &= q/2
\end{align*}

즉, $0$ 과 $q/4$ 는 $-q/8$ 로 $q/2$ 는 $q/8$ 로 매핑된다는 사실을 쉽게 알 수 있다.

이후 $q/8$ 을 더해주게 되면 논리 연산 결과가 0일 때는 $0$ 을 반환하고 반대로 논리 연산 결과가 1일 때는 $q/4$ 을 반환한다는 사실을 알 수 있다.

AND gate 를 위한 blind rotation 을 코드로 직접 확인해보자.

In [164]:
def AND_ACC_init(b, q, N):
    acc = np.zeros(N)

    for i in range(q//2):
        b = (b - 1) % q

        if b >= ((3 * q) // 8) and b < ((7 * q) // 8):
            acc[i] = q // 8
        else :
            acc[i] = (-q // 8) % q

    acc = Ring(N, q, acc)
    return acc

In [190]:
n = 16
q = 64
N = 32

snum = 0
fnum = 0

for i in range(1):
    m1 = rand.randint(0, 1)
    m2 = rand.randint(0, 1)
    a = np.array([rand.randint(0, q) for _ in range(n)])
    s = np.array([rand.randint(0, 1) for _ in range(n)])
    b = ((m1 * q//4 + m2 * q//4) + sum(a * s)) % q

    acc = AND_ACC_init(b, q, N)
    res = Blind_rotation(acc, a, s, q, n, N)
    constant_term = (res[0] + q//8) % q

    print("Constant term : ", constant_term, " and (m1 and m2) = ", m1 and m2)

    # if constant_term   == q//4  and (m1 and m2) == 1:
    #     snum += 1
    # elif constant_term == 0     and (m1 and m2) == 0:
    #     snum += 1
    # else:
    #     fnum += 1

# print("Success number : ", snum)
# print("Fail number    : ", fnum)

Constant term :  16  and (m1 and m2) =  1



---

## 3. LWE Extraction

위에서 정의한 blind rotation 을 통해 상수항에 우리가 원하는 값이 오도록 만들었다.

그러나 지금 $\textsf{acc}$ 는 엄연히 RLWE 암호문이다. 즉, 우리가 메세지를 암호활 때 사용한 LWE 와 다른 상황이다.

이러한 RLWE 의 상수항을 다시 LWE 로 만들어주는 과정을 LWE extraction 이라고 하며 이는 다음과 같이 정의된다.

>$\textbf{Definition : LWE extraction}$
>
>RLWE 암호문 $(\boldsymbol{a}, \boldsymbol{b})$ 이 주어졌다고 하자. 이 때 상수항을 추출하는 LWE extraction 은 다음과 같이 정의된다:
>
>\begin{align*}
>    \textsf{LWE\_Ext}((\boldsymbol{a}, \boldsymbol{b})) = (
>        (\boldsymbol{a}_0, -\boldsymbol{a}_{N-1}, -\boldsymbol{a}_{N-2}, \dots, -\boldsymbol{a}_{1}), \boldsymbol{b}_0) \in \mathbb{Z}_Q^{N+1}
>\end{align*}
>
>이 때 이 암호문에 대한 비밀키는 $(\boldsymbol{s}_0, \dots, \boldsymbol{s}_{N-1}) \in \mathbb{Z}_Q^N$ 이다.

약간 이상한 꼴로 정의되어 있다는 것을 알 수 있다.

그 이유는 실제로 RLWE 암호문을 복호화 할 때 사용하는 $\boldsymbol{a} \cdot \boldsymbol{s}$ 가 negacyclic convolution 이기 때문이다.

코드를 통해 확인해보자.

In [243]:
def LWEextract(ctxt:RLWEctxt) -> LWEctxt:
    a, b = ctxt
    b_0  = int(b.coeffs[0])                                
    a_coeffs = [a.coeffs[0]] + [(-x) % a.q for x in reversed(a.coeffs[1:])]

    return (a_coeffs, b_0)

In [265]:
n = 16
N = 256
q = 128

lwe  = LWE(n, q, std=0)
rlwe = RLWE(N, q, std=0)

s         = lwe.keygen()
s_ring, _ = rlwe.keygen()

m            = Ring(N, q, [q//2 - i for i in range(N)])
ct           = rlwe.encrypt(m, s_ring)
extracted_ct = LWEextract(ct)

pt = lwe.decrypt(extracted_ct, np.array(s_ring.coeffs))

print("RLWE msg  : ", m)
print("Plaintext : ", pt)

RLWE msg  :  (64 + 63x + 62x^2 + 61x^3 + 60x^4 + 59x^5 + 58x^6 + 57x^7 + 56x^8 + 55x^9 + 54x^10 + 53x^11 + 52x^12 + 51x^13 + 50x^14 + 49x^15 + 48x^16 + 47x^17 + 46x^18 + 45x^19 + 44x^20 + 43x^21 + 42x^22 + 41x^23 + 40x^24 + 39x^25 + 38x^26 + 37x^27 + 36x^28 + 35x^29 + 34x^30 + 33x^31 + 32x^32 + 31x^33 + 30x^34 + 29x^35 + 28x^36 + 27x^37 + 26x^38 + 25x^39 + 24x^40 + 23x^41 + 22x^42 + 21x^43 + 20x^44 + 19x^45 + 18x^46 + 17x^47 + 16x^48 + 15x^49 + 14x^50 + 13x^51 + 12x^52 + 11x^53 + 10x^54 + 9x^55 + 8x^56 + 7x^57 + 6x^58 + 5x^59 + 4x^60 + 3x^61 + 2x^62 + 1x^63 + 0x^64 + 127x^65 + 126x^66 + 125x^67 + 124x^68 + 123x^69 + 122x^70 + 121x^71 + 120x^72 + 119x^73 + 118x^74 + 117x^75 + 116x^76 + 115x^77 + 114x^78 + 113x^79 + 112x^80 + 111x^81 + 110x^82 + 109x^83 + 108x^84 + 107x^85 + 106x^86 + 105x^87 + 104x^88 + 103x^89 + 102x^90 + 101x^91 + 100x^92 + 99x^93 + 98x^94 + 97x^95 + 96x^96 + 95x^97 + 94x^98 + 93x^99 + 92x^100 + 91x^101 + 90x^102 + 89x^103 + 88x^104 + 87x^105 + 86x^106 + 85x^107 + 84x


---

이후에는 $\boldsymbol{s}$ 로 정의되어 있는 비밀키를 key switching 을 통해서 다시 $\vec{s}$ 로 돌려둔 뒤에 연산을 이어나간다.